# Langchain crash course

In [1]:
# from secret_key import openapi_key
# import os
# os.environ['OPENAI_API_KEY'] = openapi_key

import os
os.environ['OPENAI_API_KEY'] = "YOUR OPEN AI KEY"

## LLMs

In [6]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.6) #temperature is how creative / "risky" you want it to be
name = llm.invoke("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)



"Spice Kingdom"


In [8]:
llm.invoke("I want to open a restaurant for Indian food. Suggest a fency name for this.")

'\n\n"Maharaja\'s Palace"'

## Prompt Templates

In [10]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)



I want to open a restaurant for Italian food. Suggest a fency name for this.


In [11]:
print(llm.predict(p))



"La Dolce Vita Trattoria" (The Sweet Life Trattoria)


## Chains

In [15]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Mexican")

'\n\n"El Sabor Exquisito" (The Exquisite Flavor) '

In [16]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run("Mexican")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


'\n"Cantina del Sol" '

In [17]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [18]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Indian")
print(content)

 

1. Spicy Chicken Tikka Masala: Tender pieces of chicken cooked in a spicy tomato-based sauce, served with basmati rice and naan bread. 

2. Veggie Samosas: Crispy pastry pockets filled with spiced potatoes, peas, and vegetables, served with mint chutney for dipping. 

3. Lamb Vindaloo: A fiery dish of tender lamb pieces cooked in a spicy sauce with potatoes, served with fragrant basmati rice. 

4. Paneer Tikka Masala: Cubes of homemade cottage cheese cooked in a spicy tomato and cream sauce, served with naan bread. 

5. Tandoori Shrimp: Marinated shrimp cooked in a traditional clay oven, served with a side of spicy mint chutney. 

6. Aloo Gobi: A classic vegetarian dish of potatoes and cauliflower cooked with aromatic spices and served with basmati rice. 

7. Chicken Korma: Tender chicken pieces cooked in a creamy coconut and cashew sauce, served with fragrant basmati rice. 

8. Dal Makhani: A rich and creamy lentil dish, slow-cooked with spices and served with naan bread. 

9. Tand

#### Sequential Chain

In [19]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [20]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [21]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [22]:
chain({"cuisine": "Indian"})

/Users/rohansaxena/Desktop/AI Engineer/Week 31-32 - LLM & Langchain/langchain_crash_course/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'cuisine': 'Indian',
 'restaurant_name': '\n\n"Maharaja\'s Delight"',
 'menu_items': '\n\n1. Chicken Tikka Masala\n2. Lamb Rogan Josh\n3. Vegetable Korma\n4. Palak Paneer (spinach and cottage cheese curry)\n5. Tandoori Chicken\n6. Butter Naan\n7. Biryani (chicken or vegetable)\n8. Malai Kofta (vegetable dumplings in creamy tomato sauce)\n9. Dal Makhani (creamy black lentils)\n10. Aloo Gobi (potato and cauliflower curry)\n11. Samosas (vegetable or meat-filled pastry)\n12. Mango Lassi (yogurt-based mango smoothie)\n13. Gulab Jamun (fried dough balls in rose syrup)\n14. Rasmalai (soft cheese balls in sweetened milk)\n15. Masala Chai (spiced tea)'}

## Agents

In [26]:
# make sure yoy have installed this package: pip install google-search-results
# from secret_key import serpapi_key
# os.environ['SERPAPI_API_KEY'] = serpapi_key

os.environ['SERPAPI_API_KEY'] = "4f8cddb3037b068ab01ec61e7f3dfe56656c104fa8bf08ebf5efc4b9ee5a6e30"

#### serpapi and llm-math tool

In [27]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What was the GDP of US in 2022 plus 5?")



> Entering new AgentExecutor chain...
 I should use the Calculator tool to solve this math problem.
Action: Calculator
Action Input: 2022 + 5
Observation: Answer: 2027
Thought: Now I need to search for the GDP of the US in 2027.
Action: Search
Action Input: "GDP of US in 2027"
Observation: {'type': 'organic_result', 'title': 'United States - Gross domestic product (GDP) 2029 - Statista', 'source': 'Statista'}
Thought: I now know the final answer.
Final Answer: The GDP of the US in 2027 is $24.5 trillion.

> Finished chain.


'The GDP of the US in 2027 is $24.5 trillion.'

#### Wikipedia and llm-math tool

In [29]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his age right now in 2023?")



> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about Elon Musk's birthdate and age.
Action: wikipedia
Action Input: "Elon Musk"
Observation: Page: Elon Musk
Summary: Elon Reeve Musk (; born June 28, 1971) is a businessman and investor. He is best known for his key roles in space company SpaceX and automotive company Tesla, Inc. Other notable involvements include ownership of X Corp., formerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI. He is one of the wealthiest people in the world; as of June 2024, Forbes estimates his net worth to be US$210 billion.
A member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania and 

'Elon Musk will be 52 years old in 2023.'

## Memory

In [30]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



"El Sabor Exquisito" (The Exquisite Flavor)


In [31]:
name = chain.run("Indian")
print(name)



"Spice Palace"


In [32]:
chain.memory

In [33]:
type(chain.memory)

NoneType

#### ConversationBufferMemory

In [34]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)



"El Sabor Exquisito" (The Exquisite Flavor)


In [35]:
name = chain.run("Arabic")
print(name)



"Al-Fawarish" (meaning "The Treasures" in Arabic)


In [36]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Sabor Exquisito" (The Exquisite Flavor)
Human: Arabic
AI: 

"Al-Fawarish" (meaning "The Treasures" in Arabic)


#### ConversationChain

In [37]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [38]:
convo.run("Who won the first cricket world cup?")

" The first cricket world cup was won by West Indies in 1975. The final match was played between West Indies and Australia at Lord's Cricket Ground in London, England. West Indies won by 17 runs, with Clive Lloyd being named the player of the match for his score of 102 runs off 85 balls."

In [39]:
convo.run("How much is 5+5?")

' The sum of 5 and 5 is 10.'

In [40]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the winning team, West Indies, was Clive Lloyd. He was also named the player of the match for his outstanding performance.'

In [41]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was won by West Indies in 1975. The final match was played between West Indies and Australia at Lord's Cricket Ground in London, England. West Indies won by 17 runs, with Clive Lloyd being named the player of the match for his score of 102 runs off 85 balls.
Human: How much is 5+5?
AI:  The sum of 5 and 5 is 10.
Human: Who was the captain ofthe winning team?
AI:  The captain of the winning team, West Indies, was Clive Lloyd. He was also named the player of the match for his outstanding performance.


#### ConversationBufferWindowMemory

In [42]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

' The first cricket world cup was won by West Indies in 1975. It was held in England and was a 60-over tournament. The West Indies team defeated Australia in the final match by 17 runs. It was a historic moment for the West Indies as they became the first team to win the cricket world cup.'

In [43]:
convo.run("How much is 5+5?")

'  The answer to 5+5 is 10. This is a basic arithmetic problem with a well-known solution. Are you familiar with the concept of addition and numerical values? If not, I can provide more information.'

In [44]:
convo.run("Who was the captain of the winning team?")

' I do not have enough context to accurately answer your question. Can you provide more information about which team you are referring to?'